## 3.1.3. Bài tập thực hành 1: Xây dựng mô hình phân cụm K-means trên tập dữ liệu chim cánh cụt

Chia dữ liệu chim cánh cụt thành $K$ nhóm riêng biệt sao cho các cá thể trong cùng nhóm có đặc điểm hình thái (mỏ, cánh, cân nặng) giống nhau nhất.
Quy trình thực hiện:Tiền xử lý: Làm sạch dữ liệu (dropna) và Quan trọng nhất là Chuẩn hóa dữ liệu (StandardScaler) để đưa các đơn vị đo khác nhau (mm, gram) về cùng một hệ quy chiếu.
Tìm K: Sử dụng phương pháp Elbow (Khuỷu tay). Kết quả cho thấy tại $K=3$, độ lỗi giảm đáng kể nhất (điểm gập), gợi ý có 3 loài chính.
Phân cụm: Chạy thuật toán với $K=3$.Trực quan hóa: Vẽ biểu đồ phân tán để xem ranh giới giữa các cụm.

In [ ]:
plt.figure(figsize=(12, 8))

# Hàm vẽ chính
plt.figure(figsize=(10, 6))

sns.scatterplot(
    data=df_clean,
    x='culmen_length_mm',  # Trục X
    y='culmen_depth_mm',   # Trục Y
    hue='Cluster',         # MÀU SẮC: Phân biệt Cụm (0, 1, 2)
    style='sex',           # HÌNH DÁNG: Phân biệt Giới tính (MALE, FEMALE)
    palette='viridis',     # Bảng màu
    s=100,                 # Kích thước điểm
    alpha=0.8              # Độ trong suốt
)

plt.title('Phân cụm K-means: Màu theo Cụm - Hình dáng theo Giới tính', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

*Đoạn code vẽ biểu đồ phân tán kết hợp đa chiều thông tin (Vừa xem cụm, vừa xem giới tính trên nền 2D).*

**Tại sao lại dùng biểu đồ như vậy:**

Thông thường biểu đồ 2D chỉ hiện được 2 thông tin (trục X, Y).

Đoạn code này giúp chúng ta nhìn thấy 4 chiều thông tin cùng lúc: Chiều dài mỏ (X), Độ sâu mỏ (Y), Cụm (Màu), và Giới tính (Hình dáng).

Nó giúp trả lời ngay câu hỏi khó: "Liệu các cụm này có bị chi phối bởi giới tính không?" mà không cần vẽ thêm biểu đồ khác.

## **Kết luận:**

Mô hình K-means đã chứng minh tính hiệu quả cao trong việc phân loại sơ bộ dữ liệu chim cánh cụt.Về mặt kỹ thuật: 
Phương pháp Elbow cho tín hiệu rất rõ ràng tại $K=3$, khẳng định dữ liệu có cấu trúc phân tách tự nhiên mạnh mẽ, không bị chồng chéo quá nhiều.

Về mặt dữ liệu: Kết quả phân cụm cho thấy 3 nhóm này có đặc điểm hình thái riêng biệt: một nhóm có mỏ ngắn-sâu, một nhóm mỏ dài-nông và một nhóm mỏ dài-sâu trung bình.

Phát hiện thú vị: Khi kết hợp với thông tin giới tính (biểu đồ highlight), ta thấy sự phân bố xen kẽ giữa đực và cái trong cùng một cụm. Điều này dẫn đến kết luận quan trọng: Sự khác biệt về kích thước giữa các loài (Cụm) lớn hơn nhiều so với sự khác biệt giới tính. Tức là, dù là con đực hay cái thì đặc điểm loài vẫn là yếu tố chính quyết định hình dáng cơ thể.

---

## 3.2.3. Bài tập thực hành 1: Xây dựng mô hình phân cụm đa cấp trên tập dữ liệu chim cánh cục.

Xây dựng một cấu trúc phân cấp (như cây phả hệ) cho dữ liệu chim cánh cụt, từ đó tìm ra các nhóm tự nhiên mà không cần đoán trước số cụm. 
Quy trình thực hiện:Tiền xử lý: Tương tự K-means (Chuẩn hóa là bắt buộc).
Xây dựng cây: Tính toán ma trận khoảng cách và vẽ Dendrogram.Cắt cây (Cutting the tree): Dựa vào dendrogram, chọn điểm cắt tại mức khoảng cách phù hợp (tạo ra 3 nhánh lớn) $\rightarrow$ Chọn 3 cụm. 
Phân tích sâu: Sử dụng AgglomerativeClustering để gán nhãn và vẽ Clustermap/Boxplot để so sánh đặc tính các nhóm.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

features = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']

X_scaled_df = pd.DataFrame(X_scaled, columns=features, index=df_clean.index)

# Tạo bảng màu để đánh dấu các cụm
cluster_colors = df_clean['Cluster_Agg'].map({0: 'purple', 1: 'green', 2: 'yellow'})

# --- VẼ BIỂU ĐỒ ---
plt.figure(figsize=(10, 10))
g = sns.clustermap(
    X_scaled_df,
    method='ward', 
    cmap='viridis',   
    row_colors=cluster_colors, 
    figsize=(10, 12),
    dendrogram_ratio=(.1, .2), 
    cbar_pos=(0, .2, .03, .4)   
)

g.fig.suptitle('Biểu đồ Clustermap: Quan hệ giữa các mẫu và đặc trưng', y=1.02)
plt.show()

_Đoạn code vẽ Clustermap (Biểu đồ nhiệt kèm cây phân cấp)._

**Tại sao biểu đồ này đặc biệt**

Đây là một biểu đồ "quyền lực" trong Data Science. Nó không chỉ gom nhóm các mẫu (hàng) mà còn gom nhóm các đặc trưng (cột).

Nhìn vào biểu đồ này, bạn thấy ngay logic của máy tính: "Tại sao nhóm này lại màu Vàng?" $\rightarrow$ Vì cột body_mass_g và flipper_length_mm của nhóm đó sáng rực lên.

Nó giải thích được "Tại sao" (Why) chứ không chỉ là "Cái gì" (What) như biểu đồ Scatter plot thông thường.

## **Kết luận:**

So với K-means, mô hình phân cụm đa cấp cung cấp cái nhìn "trong suốt" và chi tiết hơn về cấu trúc dữ liệu.

Về mặt cấu trúc: Biểu đồ Dendrogram không chỉ xác nhận con số $K=3$ là chính xác mà còn cho thấy khoảng cách giữa các nhóm. Nó chỉ ra rằng có một nhóm (Cluster 1 - Nhóm Cánh dài) tách biệt hoàn toàn so với hai nhóm còn lại ngay từ những nhánh đầu tiên của cây phả hệ.

Về mặt đặc trưng (Feature Insight): Thông qua Clustermap (phần code highlight), ta phát hiện ra sự tương quan mạnh mẽ (cùng sáng màu) giữa Cân nặng (Body Mass) và Chiều dài cánh (Flipper Length). Điều này giải thích tại sao nhóm chim "khổng lồ" lại được tách ra riêng: chúng không chỉ nặng hơn mà cấu trúc cánh cũng phát triển vượt trội để phù hợp với cơ thể.

Về ngoại lai: Mô hình này cực kỳ nhạy bén trong việc phát hiện điểm ngoại lai (outlier), thể hiện qua việc nó tách riêng 1 mẫu dữ liệu bất thường thành một nhánh hoặc cụm riêng nếu ta cắt cây ở mức thấp hơn.